In [1]:
import pandas as pd

df = pd.read_csv(r'D:\Users\paulo\PycharmProjects\fastcamp-agent\card13\mtsamples.csv')



In [2]:
df.head(-1)

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."
...,...,...,...,...,...,...
4993,4993,"Chronic glossitis, xerostomia, probable envir...",Allergy / Immunology,Evaluation of Allergies,"HISTORY:, A 55-year-old female presents self-...","allergy / immunology, chronic glossitis, xeros..."
4994,4994,Patient having severe sinusitis about two to ...,Allergy / Immunology,Chronic Sinusitis,"HISTORY:, I had the pleasure of meeting and e...",NaN
4995,4995,This is a 14-month-old baby boy Caucasian who...,Allergy / Immunology,Kawasaki Disease - Discharge Summary,"ADMITTING DIAGNOSIS: , Kawasaki disease.,DISCH...","allergy / immunology, mucous membranes, conjun..."
4996,4996,A female for a complete physical and follow u...,Allergy / Immunology,Followup on Asthma,"SUBJECTIVE: , This is a 42-year-old white fema...",NaN


In [3]:
df['transcription'][5]

'PREOPERATIVE DIAGNOSIS: , Morbid obesity.,POSTOPERATIVE DIAGNOSIS:  ,Morbid obesity.,PROCEDURE: , Laparoscopic antecolic antegastric Roux-en-Y gastric bypass with EEA anastomosis.,ANESTHESIA: , General with endotracheal intubation.,INDICATION FOR PROCEDURE: , This is a 30-year-old female, who has been overweight for many years.  She has tried many different diets, but is unsuccessful.  She has been to our Bariatric Surgery Seminar, received some handouts, and signed the consent.  The risks and benefits of the procedure have been explained to the patient.,PROCEDURE IN DETAIL:  ,The patient was taken to the operating room and placed supine on the operating room table.  All pressure points were carefully padded.  She was given general anesthesia with endotracheal intubation.  SCD stockings were placed on both legs.  Foley catheter was placed for bladder decompression.  The abdomen was then prepped and draped in standard sterile surgical fashion.  Marcaine was then injected through umbili

In [4]:
df.iloc[0]

Unnamed: 0                                                           0
description           A 23-year-old white female presents with comp...
medical_specialty                                 Allergy / Immunology
sample_name                                         Allergic Rhinitis 
transcription        SUBJECTIVE:,  This 23-year-old white female pr...
keywords             allergy / immunology, allergic rhinitis, aller...
Name: 0, dtype: object

In [5]:
import os
from dotenv import load_dotenv


load_dotenv()
# os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
# os.environ['GEMINI_API_KEY'] = os.getenv('GEMINI_API_KEY')

In [6]:
from crewai import Crew, Process, Agent, Task, LLM
from langchain_openai import ChatOpenAI

In [7]:
# llm_OpenAI = ChatOpenAI()
llm_groq_llama = LLM(model="groq/llama-3.3-70b-versatile", api_key=os.environ["GROQ_API_KEY"])
llm_groq_deepseek = LLM(model="groq/deepseek-r1-distill-llama-70b", api_key=os.environ["GROQ_API_KEY"])
llm_groq_compound= LLM(model="groq/compound-beta", api_key=os.environ["GROQ_API_KEY"])

# llm_gemini = LLM(model='gemini/gemini-1.5-flash', api_key=os.environ["GEMINI_API_KEY"])


In [8]:

medico_generalista = Agent(
    role='Médico generalista',
    backstory='''Você é um médico capaz de analisar dados textuais de pacientes para classificar a possível especialidade e modalidade médica.''',
    goal="Classificar especialidade e modalidade médica.",
    llm=llm_groq_llama
)

medico_laudador = Agent(
    role='Médico laudador',
    backstory='''Você é um médico laudador que tem a função de elaborar laudos médicos, avaliações clínicas e de informar a especialidade e modalidade médica.''',
    goal="Classificar especialidade e modalidade médica.",
    llm=llm_groq_deepseek
)

medico_perito = Agent(
    role='Médico perito',
    backstory='''Um perito médico é um profissional da área da saúde que atua como especialista, fornecendo conhecimento técnico-científico para auxiliar na resolução de questões judiciais ou administrativas que envolvem saúde.''',
    goal="Classificar especialidade e modalidade médica.",
    llm=llm_groq_compound
)

In [9]:
from textwrap import dedent

def get_task(transcricao, agent):
  return Task(description=dedent(f"""\
    Entender os dados textuais de entrada e classificar a transcrição médica. Primeiro classificar a <especialidade> e depois a <modalidade> conforme as listas a seguir.

      <especialidade>
      Abdome;
      Cabeça e Pescoço;
      Cardio;
      Mama;
      Radiologia Intervencionista;
      Medicina Nuclear;
      Musculoesquelético;
      Neurologia;
      Tórax;
      Ultrassonografia;
      </especialidade>.

      <modalidade>
      Clínica Médica;
      Pediatria;
      Cirurgia Geral;
      Ginecologia e Obstetrícia;
      Neurologia;
      Psiquiatria;
      Cardiologia;
      Endocrinologia;
      Dermatologia;
      Oftalmologia;
      Otorrinolaringologia;
      Ortopedia e Traumatologia;
      Pneumologia
      Radiologia e Diagnóstico por Imagem;
      Outros;
      </modalidade>.


      Aqui está a transcrição médica utilizada como dado textual de entrada para ser classificado:

      <transcrição>
      {transcricao}
      </transcrição>

      """),
    agent=agent, expected_output='Retorne a especialidade e depois a modalidade médica separados por vírgula, por exemplo: Mamografia, Mama'
  )


In [14]:
df_output_list = []
df['generalista_especialidade'] = None
df['laudador_especialidade'] = None
df['perito_especialidade'] = None

df['generalista_modalidade'] = None
df['laudador_modalidade'] = None
df['perito_modalidade'] = None



for index, row in df.iterrows():
    try:


        print('############################################## '+ str(index))
        transcricao_clinica = f'''
          Descrição clínica: {row['description']}.
          Transcrição clínica: {row['transcription']}.
          Palavras-chave: {row['keywords']}.
          '''

      # definindo a tarefa genérica para generalista
        task_generalista = get_task(transcricao_clinica, medico_generalista)

        crew_generalista = Crew(
        agents=[medico_generalista],
        tasks=[task_generalista],
        verbose=False,
        memory=True,
        process=Process.sequential
      )
        result_generalista = crew_generalista.kickoff()


        # definindo a tarefa genérica para laudador
        task_laudador = get_task(transcricao_clinica, medico_laudador)

        crew_laudador = Crew(
        agents=[medico_laudador],
        tasks=[task_laudador],
        verbose=False,
        memory=True,
        process=Process.sequential
      )
        result_laudador = crew_laudador.kickoff()


       # definindo a tarefa genérica para perito
        task_perito = get_task(transcricao_clinica, medico_perito)

        crew_perito = Crew(
        agents=[medico_perito],
        tasks=[task_perito],
        verbose=False,
        memory=True,
        process=Process.sequential
      )
        result_perito = crew_perito.kickoff()

        especialidade_modalidade = result_generalista.raw.split(",")
        df.at[index,'generalista_especialidade'] = especialidade_modalidade[0]
        df.at[index,'generalista_modalidade'] = especialidade_modalidade[1]

        especialidade_modalidade = result_laudador.raw.split(",")
        df.at[index,'laudador_especialidade'] = especialidade_modalidade[0]
        df.at[index,'laudador_modalidade'] = especialidade_modalidade[1]


        especialidade_modalidade = result_perito.raw.split(",")
        df.at[index,'perito_especialidade'] = especialidade_modalidade[0]
        df.at[index,'perito_modalidade'] = especialidade_modalidade[1]
    except Exception as e:
        print(e)


############################################## 0
############################################## 1
############################################## 2
############################################## 3
############################################## 4
############################################## 5
############################################## 6
############################################## 7
############################################## 8
############################################## 9
############################################## 10
############################################## 11
############################################## 12
############################################## 13
############################################## 14
############################################## 15
############################################## 16
############################################## 17
############################################## 18
############################################## 19
##########

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `compound-beta` in organization `org_01jhc3pgw2ema9vx7xrt725fda` service tier `on_demand` on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 6m32.45s. Need more tokens? Visit https://groq.com/self-serve-support/ to request higher limits.","type":"requests","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `compound-beta` in organization `org_01jhc3pgw2ema9vx7xrt725fda` service tier `on_demand` on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 6m32.45s. Need more tokens? Visit https://groq.com/self-serve-support/ to request higher limits.","type":"requests","code":"rate_limit_exceeded"}}

 An unknown error occurred. Please check the details below.
 Error details: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `compound-beta` in organization `org_01jhc3pgw2ema9vx7xrt725fda` service tier `on_demand` on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 6m32.45s. Need more tokens? Visit https://groq.com/self-serv

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `compound-beta` in organization `org_01jhc3pgw2ema9vx7xrt725fda` service tier `on_demand` on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 6m15.715s. Need more tokens? Visit https://groq.com/self-serve-support/ to request higher limits.","type":"requests","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `compound-beta` in organization `org_01jhc3pgw2ema9vx7xrt725fda` service tier `on_demand` on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 6m15.715s. Need more tokens? Visit https://groq.com/self-serve-support/ to request higher limits.","type":"requests","code":"rate_limit_exceeded"}}

 An unknown error occurred. Please check the details below.
 Error details: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `compound-beta` in organization `org_01jhc3pgw2ema9vx7xrt725fda` service tier `on_demand` on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 6m15.715s. Need more tokens? Visit https://groq.com/self-se

...

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `compound-beta` in organization `org_01jhc3pgw2ema9vx7xrt725fda` service tier `on_demand` on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 48.176999999s. Need more tokens? Visit https://groq.com/self-serve-support/ to request higher limits.","type":"requests","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `compound-beta` in organization `org_01jhc3pgw2ema9vx7xrt725fda` service tier `on_demand` on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 48.176999999s. Need more tokens? Visit https://groq.com/self-serve-support/ to request higher limits.","type":"requests","code":"rate_limit_exceeded"}}

 An unknown error occurred. Please check the details below.
 Error details: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `compound-beta` in organization `org_01jhc3pgw2ema9vx7xrt725fda` service tier `on_demand` on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 48.176999999s. Need more tokens? Visit https://groq.com

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `compound-beta` in organization `org_01jhc3pgw2ema9vx7xrt725fda` service tier `on_demand` on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 26.734999999s. Need more tokens? Visit https://groq.com/self-serve-support/ to request higher limits.","type":"requests","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `compound-beta` in organization `org_01jhc3pgw2ema9vx7xrt725fda` service tier `on_demand` on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 26.734999999s. Need more tokens? Visit https://groq.com/self-serve-support/ to request higher limits.","type":"requests","code":"rate_limit_exceeded"}}

 An unknown error occurred. Please check the details below.
 Error details: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `compound-beta` in organization `org_01jhc3pgw2ema9vx7xrt725fda` service tier `on_demand` on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 26.734999999s. Need more tokens? Visit https://groq.com

KeyboardInterrupt: 

In [18]:
df.head(200)
df.to_csv('saida.csv')